*Смирнова Александра, БКЛ162*

### Устанавливаем UDPipe

In [0]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe
!mv russian-syntagrus-ud-2.4-190531.udpipe?sequence=74 russian-syntagrus-ud-2.4-190531.udpipe
!pip install ufal.udpipe

--2019-12-25 15:38:59--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45859472 (44M) [application/octet-stream]
Saving to: ‘russian-syntagrus-ud-2.4-190531.udpipe.1’

russian-syntagrus-u 100%[===================>]  43.73M  14.3MB/s    in 3.6s    

2019-12-25 15:39:04 (12.0 MB/s) - ‘russian-syntagrus-ud-2.4-190531.udpipe.1’ saved [45859472/45859472]

mv: cannot stat 'russian-syntagrus-ud-2.4-190531.udpipe?sequence=74': No such file or directory


In [0]:
import ufal.udpipe

class UDPipeModel:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [0]:
model = UDPipeModel('russian-syntagrus-ud-2.4-190531.udpipe')
sentences = model.tokenize('Вставай, проклятьем заклейменный!')
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")
print(conllu)

# newdoc
# newpar
# sent_id = 1
# text = Вставай, проклятьем заклейменный!
1	Вставай	вставать	VERB	_	Aspect=Imp|Mood=Imp|Number=Sing|Person=2|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
2	,	,	PUNCT	_	_	3	punct	_	_
3	проклятьем	проклятьть	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin|Voice=Act	1	conj	_	_
4	заклейменный	заклейметь	VERB	_	Aspect=Perf|Case=Nom|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Pass	3	xcomp	_	SpaceAfter=No
5	!	!	PUNCT	_	_	1	punct	_	SpaceAfter=No




### Загружаем и обрабатываем .тхт-файл

In [0]:
!wget 'https://raw.githubusercontent.com/SashaSlon/NLP_homeworks_2019-20/master/hw2/testset2.txt'

--2019-12-25 15:40:39--  https://raw.githubusercontent.com/VictoriaRe/NLP-course/master/testset2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1894767 (1.8M) [text/plain]
Saving to: ‘testset2.txt’

testset2.txt        100%[===================>]   1.81M  --.-KB/s    in 0.08s   

2019-12-25 15:40:39 (23.3 MB/s) - ‘testset2.txt’ saved [1894767/1894767]



In [0]:
with open('testset2.txt', 'r', encoding='utf-8') as src:
    text = src.read()

In [0]:
model = UDPipeModel('russian-syntagrus-ud-2.4-190531.udpipe')
sentences = model.tokenize(text)
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")


In [0]:
with open('testset2.conllu', 'w', encoding='utf-8') as f:
    f.write(conllu)

In [0]:
with open('testset2.conllu', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

12435312


**1. Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).**

In [0]:
from nltk.parse import DependencyGraph

In [0]:
trees = []

with open('testset2.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    
    for sent in parsed_sents:
        tree = []

        for string in sent.split('\n'):
            if string and string[0] != '#':
                tree.append(string)

        trees.append('\n'.join(tree))

In [0]:
def get_subtree(nodes, node, rel_needed):
    res=[]
    if 'deps' in nodes[node]:
        for relation in nodes[node]['deps']:
            if relation in rel_needed:
                for dep in nodes[node]['deps'][relation]:
                    res.append(dep)
    return res     

                          

In [0]:
def flatten(l):
    flat = []

    for el in l:
        if isinstance(el, list):
            flat += flatten(el)
            
        else:
            flat.append(el)

    return flat

In [0]:
result = []
verbs_count = {}
verbs = []
error_list = []
N=0

for tree in trees:
  try:
    dg = DependencyGraph(tree, top_relation_label='root')
    np_list = []

    for n in dg.nodes:
        if dg.nodes[n]['ctag'] == 'VERB':
          verb=dg.nodes[n]['word'].lower()
          verbs.append(verb)
          
          np = list(sorted(get_subtree(dg.nodes, n, ['obj'])))
          for i in np:
            w2=dg.nodes[n]['word']+' '+dg.nodes[i]['word']
            np_list.append((dg.nodes[n]['word'], dg.nodes[i]['word']))
    
    if len(np_list)>0:
      for el in np_list:
        result.append(el)

  except:
    err_line = 'ERROR:' + tree
    error_list.append(err_line)

result

[('заявил', 'Ъ'),
 ('подал', 'иск'),
 ('признать', 'договор'),
 ('запускать', 'механизм'),
 ('рассмотрит', 'вопрос'),
 ('отстранил', 'его'),
 ('терпящих', 'бедствие'),
 ('определят', 'срок'),
 ('обвинили', 'которого'),
 ('оберегали', 'покой'),
 ('обвинили', 'руководство'),
 ('вызвали', 'недовольство'),
 ('привело', 'обитателей'),
 ('гласит', 'постановление'),
 ('приносила', 'пользу'),
 ('задержали', 'его'),
 ('уберегло', 'его'),
 ('получили', 'сроки'),
 ('собиравший', 'доказательства'),
 ('обвинивший', 'олигарха'),
 ('предали', 'суду'),
 ('предали', 'роту'),
 ('оплачивать', 'обязанность'),
 ('обвинил', 'Юлию'),
 ('Выиграть', 'суд'),
 ('направило', 'ведомства'),
 ('девоенизировать', 'суды'),
 ('имеет', '|'),
 ('просил', 'суд'),
 ('освободить', 'его'),
 ('сделал', 'заявление'),
 ('рассматривал', 'ходатайство'),
 ('признал', '|'),
 ('затрудняет', 'доступ'),
 ('отказал', 'жалобы'),
 ('отозвала', 'иск'),
 ('сделала', 'шаг'),
 ('отозвала', 'иск'),
 ('предоставлять', 'данные'),
 ('затевать', 

In [0]:
def select_pair(pair, List_verbs):
  res=[]
  for p in pair:

    for x in p:
      word_pair=set(x.split())

      if len(word_pair.intersection(List_verbs))>0:
        res.append(p)
     
  return res


In [0]:
from collections import Counter
verbs_counter = Counter(verbs)
verbs_more_50 = []

for w in verbs_counter.elements():
    if verbs_counter[w] >= 50:
        verbs_more_50.append(w)

list_pair = select_pair(result, verbs_more_50)

verbs_more_50 = set(verbs_more_50)

list_pair

[('заявил', 'Ъ'),
 ('подал', 'иск'),
 ('признать', 'договор'),
 ('обвинили', 'которого'),
 ('обвинили', 'руководство'),
 ('обвинил', 'Юлию'),
 ('признал', '|'),
 ('отказал', 'жалобы'),
 ('обвинив', 'его'),
 ('обвинила', 'господина'),
 ('обвинила', 'Березовского'),
 ('обвинила', 'Березовского'),
 ('вынес', 'постановление'),
 ('принял', 'решение'),
 ('признать', 'ответственность'),
 ('выплатить', 'компенсации'),
 ('воспрепятствует', 'является'),
 ('вынес', 'вердикт'),
 ('обвинив', 'его'),
 ('подала', 'иск'),
 ('подтвердил', 'правомочность'),
 ('удовлетворил', 'ходатайство'),
 ('подтвердил', 'доходы'),
 ('обвинили', 'сына'),
 ('подтвердил', 'решение'),
 ('удовлетворил', 'требования'),
 ('заявил', 'Ъ'),
 ('взыскать', 'деньги'),
 ('признал', 'постановление'),
 ('удовлетворил', 'иск'),
 ('отклонил', 'иск'),
 ('приговорил', 'Березовского'),
 ('вынес', 'приговор'),
 ('подтвердил', 'приказ'),
 ('обвинил', 'Киселева'),
 ('подал', 'иск'),
 ('заявил', 'ходатайство'),
 ('удовлетворил', 'иск'),
 ('п

In [0]:
!pip install pymorphy2

In [0]:
from pymorphy2 import MorphAnalyzer

m = MorphAnalyzer()

In [0]:
verb_noun_norm=[]

for p in list_pair:
  if len(p[0]) > 1 and len(p[1]) > 1: 
    el1=m.parse(p[0])[0].normal_form
    el2=m.parse(p[1])[0].normal_form

    norm_pair=(el1,el2) #created normalized pair of verb+noun
    verb_noun_norm.append(norm_pair)

len(verb_noun_norm)

1041

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**2. Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).**

In [0]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [0]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens if t not in stop_words]
    return lemmas

In [0]:
import nltk
from nltk import collocations 

In [0]:
with open('testset2.txt', 'r', encoding='utf-8') as f: #, encoding='utf-8'
    testset2 = f.read()
len(testset2)

1037635

In [0]:
finder_test = collocations.BigramCollocationFinder.from_documents([normalize(testset2)])

In [0]:
finder_test.apply_ngram_filter(lambda w1, w2: (w1, w2) not in verb_noun_norm)

###Dice

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().dice)

[(('удовлетворить', 'ходатайство'), 0.17834394904458598),
 (('удовлетворить', 'иск'), 0.15155615696887687),
 (('подать', 'иск'), 0.1511216056670602),
 (('подтвердить', 'законность'), 0.12598425196850394),
 (('вынести', 'приговор'), 0.11671087533156499),
 (('выплатить', 'компенсация'), 0.10687022900763359),
 (('принять', 'решение'), 0.0903954802259887),
 (('подать', 'апелляция'), 0.08253968253968254),
 (('вынести', 'решение'), 0.08125577100646353),
 (('отклонить', 'жалоба'), 0.08088235294117647),
 (('отклонить', 'апелляция'), 0.07476635514018691),
 (('вынести', 'постановление'), 0.07246376811594203),
 (('удовлетворить', 'жалоба'), 0.06451612903225806),
 (('выплатить', 'штраф'), 0.06201550387596899),
 (('выплатить', 'иркутянин'), 0.06060606060606061),
 (('удовлетворить', 'требование'), 0.06),
 (('признать', 'правота'), 0.05649717514124294),
 (('вынести', 'вердикт'), 0.056338028169014086),
 (('отклонить', 'иск'), 0.053208137715179966),
 (('приговорить', 'саддам'), 0.047619047619047616),
 

In [0]:
dice_100 = finder_test.nbest(collocations.BigramAssocMeasures().dice, 100)
dice_100

[('удовлетворить', 'ходатайство'),
 ('удовлетворить', 'иск'),
 ('подать', 'иск'),
 ('подтвердить', 'законность'),
 ('вынести', 'приговор'),
 ('выплатить', 'компенсация'),
 ('принять', 'решение'),
 ('подать', 'апелляция'),
 ('вынести', 'решение'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'апелляция'),
 ('вынести', 'постановление'),
 ('удовлетворить', 'жалоба'),
 ('выплатить', 'штраф'),
 ('выплатить', 'иркутянин'),
 ('удовлетворить', 'требование'),
 ('признать', 'правота'),
 ('вынести', 'вердикт'),
 ('отклонить', 'иск'),
 ('приговорить', 'саддам'),
 ('принять', 'участие'),
 ('обязать', 'кашулинский'),
 ('подать', 'жалоба'),
 ('подтвердить', 'правомочность'),
 ('подтвердить', 'правомерность'),
 ('отказать', 'мосметрострой'),
 ('приговорить', 'капитан'),
 ('отклонить', 'ходатайство'),
 ('сообщить', 'адвокат'),
 ('отказать', 'сигма'),
 ('подтвердить', 'запрет'),
 ('признать', 'забастовка'),
 ('подать', 'ходатайство'),
 ('принять', 'мера'),
 ('обвинить', 'президент'),
 ('удовлетворить', 'зая

###log-likeLihood

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().likelihood_ratio)

[(('подать', 'иск'), 378.12865847466077),
 (('удовлетворить', 'иск'), 375.6688886266606),
 (('принять', 'решение'), 269.58336605197326),
 (('вынести', 'решение'), 225.39363408391418),
 (('удовлетворить', 'ходатайство'), 223.89272108263017),
 (('вынести', 'приговор'), 146.20731058697305),
 (('подать', 'апелляция'), 108.9046303732655),
 (('отклонить', 'иск'), 100.24886433413778),
 (('признать', 'правота'), 90.71186482057486),
 (('отклонить', 'жалоба'), 77.28969656667154),
 (('подтвердить', 'законность'), 75.60412679052158),
 (('удовлетворить', 'жалоба'), 64.82028574510191),
 (('вынести', 'постановление'), 63.869899117098726),
 (('выплатить', 'компенсация'), 59.775195751833365),
 (('удовлетворить', 'требование'), 51.343863431076954),
 (('признать', 'забастовка'), 48.05588248248971),
 (('вынести', 'вердикт'), 44.32209492999911),
 (('подать', 'жалоба'), 40.66395487322106),
 (('сообщить', 'адвокат'), 38.89284021833272),
 (('отклонить', 'апелляция'), 33.7502514843752),
 (('обвинить', 'президе

In [0]:
likelihood_100 = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 100)


###-PMI

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().pmi)

[(('выплатить', 'иркутянин'), 10.694561935343891),
 (('приговорить', 'взяточница'), 9.728777650681803),
 (('отказать', 'мосметрострой'), 9.43717409265124),
 (('сообщить', 'уметалиев'), 8.933010702899413),
 (('подтвердить', 'правомочность'), 8.864839200257833),
 (('обязать', 'фссна'), 8.787671339735372),
 (('подтвердить', 'готовность'), 8.186767295145195),
 (('подтвердить', 'правомерность'), 8.186767295145195),
 (('подтвердить', 'законность'), 8.016842293702881),
 (('сообщить', 'рса'), 7.933010702899411),
 (('обвинить', 'богот'), 7.818044988778892),
 (('обвинить', 'канцлер'), 7.818044988778892),
 (('обвинить', 'миллионер'), 7.818044988778892),
 (('обвинить', 'молодогвардеец'), 7.81804498877889),
 (('обязать', 'кашулинский'), 7.787671339735372),
 (('приговорить', 'водитель'), 7.7287776506818044),
 (('признать', 'хуцишвили'), 7.717282011843974),
 (('выплатить', 'дивиденд'), 7.694561935343891),
 (('признать', 'правота'), 7.454247606010183),
 (('отказать', 'сигма'), 7.437174092651238),
 (('

In [0]:
pmi_100 = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 100)
pmi_100

[('выплатить', 'иркутянин'),
 ('приговорить', 'взяточница'),
 ('отказать', 'мосметрострой'),
 ('сообщить', 'уметалиев'),
 ('подтвердить', 'правомочность'),
 ('обязать', 'фссна'),
 ('подтвердить', 'готовность'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'законность'),
 ('сообщить', 'рса'),
 ('обвинить', 'богот'),
 ('обвинить', 'канцлер'),
 ('обвинить', 'миллионер'),
 ('обвинить', 'молодогвардеец'),
 ('обязать', 'кашулинский'),
 ('приговорить', 'водитель'),
 ('признать', 'хуцишвили'),
 ('выплатить', 'дивиденд'),
 ('признать', 'правота'),
 ('отказать', 'сигма'),
 ('выплатить', 'компенсация'),
 ('отклонить', 'апелляция'),
 ('сообщить', 'морщаков'),
 ('признать', 'бабаков'),
 ('признать', 'соучастие'),
 ('обвинить', 'калягин'),
 ('подать', 'апелляция'),
 ('подать', 'кассация'),
 ('удовлетворить', 'ходатайство'),
 ('признать', 'забастовка'),
 ('подтвердить', 'план'),
 ('обвинить', 'моряк'),
 ('обвинить', 'киселев'),
 ('обвинить', 'крайком'),
 ('приговорить', 'основатель'),
 ('откло

**3. Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.**

In [0]:
gold_stand_coll=set(pmi_100).intersection(dice_100)
gold_stand_coll=set(gold_stand_coll).intersection(likelihood_100)

gold_stand_coll

{('взыскать', 'деньга'),
 ('вынести', 'вердикт'),
 ('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('выплатить', 'деньга'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'иркутянин'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('обвинить', 'моряк'),
 ('обязать', 'кашулинский'),
 ('обязать', 'структура'),
 ('обязать', 'фссна'),
 ('отказать', 'мосметрострой'),
 ('отказать', 'сигма'),
 ('отклонить', 'апелляция'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'иск'),
 ('отклонить', 'ходатайство'),
 ('подать', 'апелляция'),
 ('подать', 'иск'),
 ('подать', 'кассация'),
 ('подтвердить', 'готовность'),
 ('подтвердить', 'законность'),
 ('подтвердить', 'запрет'),
 ('подтвердить', 'информация'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'правомочность'),
 ('приговорить', 'взяточница'),
 ('приговорить', 'водитель'),
 ('приговорить', 'капитан'),
 ('приговорить', 'основатель'),
 ('приговорить', 'саддам'),
 ('признать', 'з

In [0]:
!wget 'https://raw.githubusercontent.com/SashaSlon/NLP_homeworks_2019-20/master/hw2/verb_coll.txt'

--2019-12-25 16:48:47--  https://raw.githubusercontent.com/VictoriaRe/NLP-course/master/verb_coll.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1229806 (1.2M) [text/plain]
Saving to: ‘verb_coll.txt’

verb_coll.txt       100%[===================>]   1.17M  --.-KB/s    in 0.04s   

2019-12-25 16:48:47 (27.3 MB/s) - ‘verb_coll.txt’ saved [1229806/1229806]



In [0]:
import re

with open('verb_coll.txt', 'r', encoding='utf-8') as f:
    verb_txt = f.read()

verb_coll=[]
pattern='VN.*\n'   
match = re.findall(pattern, verb_txt) 

for s in match:
    s=re.sub('VN','', s, flags=re.UNICODE)
    s = re.sub(r'\s+', ' ', s, flags=re.UNICODE)
    s=re.sub('\n','', s, flags=re.UNICODE)
    
    words = s.split()
    if len(words)==2:
        coll_lemm = (m.parse(words[0])[0].normal_form, m.parse(words[1])[0].normal_form)
        verb_coll.append(coll_lemm)


In [0]:
G_S = set(gold_stand_coll).intersection(set(verb_coll))
G_S

{('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('подать', 'иск'),
 ('принять', 'мера'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('удовлетворить', 'иск'),
 ('удовлетворить', 'требование')}